In [5]:
import pandas as pd
from ollama import chat
from ollama import ChatResponse
from litellm import completion
import litellm
import os
litellm.enable_json_schema_validation = True
litellm.set_verbose = True
import json
from litellm import ModelResponse
from dotenv import load_dotenv
load_dotenv()

# df = pd.read_json("hf://datasets/HuggingFaceH4/MATH-500/test.jsonl", lines=True)

True

In [6]:
df = pd.read_csv("100_data_math500.csv")

In [4]:
from pydantic import BaseModel
class Answer(BaseModel):
  reasoning: str
  answer: str

class AnswerCorrectness(BaseModel):
  correct:bool


In [6]:
def process_math_problems(df, start_idx:int, end_idx:int):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        response: ChatResponse = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }],
            response_format=Answer
        )
        answer_content = response.choices[0]['message']['content']

        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response'] = answer_content
        df.at[idx, 'llm_answer'] = answer
        df.at[idx, 'is_correct'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [ ]:
process_math_problems(df,start_idx=151,end_idx=300)

In [8]:
df.to_csv("300 rows gpt4o.csv")

In [11]:
def generate_training_messages(df, start_idx:int, num_rows:int):
    end_idx = min(start_idx + num_rows, len(df))
    messages = [{
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            }]
    for idx in range(start_idx, end_idx):
        row = df.iloc[idx]
        
        # Base messages that are common for all examples
        messages.extend([
            {
                'role': 'user',
                'content': row['problem']
            },
            {
                'role': 'assistant',
                'content': row['llm_raw_response']
            }
        ])
        
        # Add feedback based on correctness
        if not row['is_correct']:
            messages.append({
                'role': 'user',
                'content': f"Let me correct this. The right answer is {row['answer']}. Let's understand the solution: {row['solution']}"
            })
        else:
            messages.append({
                'role': 'user',
                'content': "Good job! Your reasoning and answer are correct!"
            })
        messages.append({
            'role': 'assistant',
            'content': "Understood. I will keep this in mind"
        })
            
       
    
    return messages

In [13]:
training_data = generate_training_messages(df,start_idx=0,num_rows=20)

In [14]:
len(training_data)

81

In [15]:
def test_process_math_problems_te(df, start_idx:int, end_idx:int):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        messages = training_data + [
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }]
        
        response = completion(model='gpt-4o', messages=messages,
            response_format=Answer
        )
        answer_content = response.choices[0]['message']['content']

        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response_test'] = answer_content
        df.at[idx, 'llm_answer_test'] = answer
        df.at[idx, 'is_correct_test'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [16]:
test_process_math_problems_te(df,start_idx=151,end_idx=300)

01:16:02 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'}, 'answer': {'title': 'Answer', 'type': 'string'}}, 'required': ['reasoning', 'answer'], 'title': 'Answer', 'type': 'object', 'additionalProperties': False}, 'name': 'Answer', 'strict': True}}, 'extra_body': {}}


01:16:06 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B1esVO89uUkgZAvhZTPJJhNnvRGte", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"The given expressions represent the squared distances between three points. Let these points be A, B, C with A and B having a distance of 3, A and C having a distance of 4, and B and C having a distance of 5. These distances correspond to a right triangle where the determinant is twice the area of the triangle. The area is \\\\( \\\\frac{1}{2} \\\\times 3 \\\\times 4 = 6 \\\\), so the square of the determinant is \\\\( (2 \\\\times 6)^2 = 144 \\\\).\",\"answer\":\"144\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739735163, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 127, "prompt_tokens": 8440, "total_tokens": 8567, "completion_tokens_det

01:16:07 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

RAW RESPONSE:
{"id": "chatcmpl-B1f0yJXvTef40qDVzzbewg26tIWT8", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"The rectangle spans from x = -4 to x = 4 (inclusive) and y = -3 to y = 3 (inclusive), totaling 9 x-coordinates and 7 y-coordinates. Since strictly inside excludes boundary points, consider integer coordinates from x = -4 to 4 and y = -3 to 3, which makes for an 8x6 grid inside. Each coordinate is an integer, so there are 8*6 inside points.\",\"answer\":\"The number of integer coordinates strictly inside the rectangle is 96.\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739735688, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 117, "prompt_tokens": 8310, "total_tokens": 8427, "completion_tokens_details": {"accepted_prediction_tokens": 0, "aud

01:24:51 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B1f10yXc2LVWRGI0I0aPvF9GTOngK", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":false}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739735690, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 6, "prompt_tokens": 125, "total_tokens": 131, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}


Processed row 299
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_format': {'type': 'json_schema', 'json_schema': {'schema': {'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'}, 'answer': {

01:24:52 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B1f11kJeYTAJHE6dkbTFiGzQCj8R5", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"reasoning\":\"The maximum number of intersection points between two circles is 2. With four distinct circles, the number of pairs of circles (nC2) is 4C2 = 6. Therefore, the maximum number of intersection points is 6 x 2 = 12.\",\"answer\":\"12\"}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739735691, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 65, "prompt_tokens": 8291, "total_tokens": 8356, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 8192}}}


SYNC kwargs[caching]: False; litellm.cache: None; kwar

In [17]:
df.to_csv('first 300 test gpt4o.csv')

In [21]:
df_test = df.iloc[20:300]

In [22]:
df_test['is_correct'].value_counts()

is_correct
True     140
False    140
Name: count, dtype: int64

In [23]:
df_test['is_correct_test'].value_counts()

is_correct_test
True     140
False    140
Name: count, dtype: int64